In [8]:
from sklearn import metrics
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.naive_bayes import MultinomialNB

from datareader import read_csv
from utils import count_false_positives_and_negatives

### Константы:
сколько строк читать, сколько % на обучающую и тестовую выборку

In [9]:
TOTAL_VALUES_COUNT = 40000
TRAIN_VALUES_PART = 0.8
TRAIN_VALUES_COUNT = int(TOTAL_VALUES_COUNT * TRAIN_VALUES_PART)

### Читаем данные из файла
И разбиваем их на обучающие и тестовые

In [10]:
df_train_test = read_csv('./data/train.csv', True, TOTAL_VALUES_COUNT)
df_train = df_train_test[:TRAIN_VALUES_COUNT]
df_test = df_train_test[TRAIN_VALUES_COUNT:]

Reading line 1000...
Reading line 2000...
Reading line 3000...
Reading line 4000...
Reading line 5000...
Reading line 6000...
Reading line 7000...
Reading line 8000...
Reading line 9000...
Reading line 10000...
Reading line 11000...
Reading line 12000...
Reading line 13000...
Reading line 14000...
Reading line 15000...
Reading line 16000...
Reading line 17000...
Reading line 18000...
Reading line 19000...
Reading line 20000...
Reading line 21000...
Reading line 22000...
Reading line 23000...
Reading line 24000...
Reading line 25000...
Reading line 26000...
Reading line 27000...
Reading line 28000...
Reading line 29000...
Reading line 30000...
Reading line 31000...
Reading line 32000...
Reading line 33000...
Reading line 34000...
Reading line 35000...
Reading line 36000...
Reading line 37000...
Reading line 38000...
Reading line 39000...
Reading line 40000...


### Формируем вектора признаков из каждой строки
Формируем разреженную матрицу априорной вероятности встретить каждое слово в имени человека/организации

In [11]:
vectorizer = HashingVectorizer(alternate_sign=False)
X_train = vectorizer.transform(df_train['name'])
X_test = vectorizer.transform(df_test['name'])

### Формируем признаки принадлежности каждой строки к классу "организация"

In [12]:
y_train = df_train['isOrg'].map(lambda v: 1 if v else 0)
y_test = df_test['isOrg'].map(lambda v: 1 if v else 0)

### Настраиваем мультиномиальный наивный Байесовский классификатор
И обучаем его

In [13]:
clf = MultinomialNB(alpha=.005)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

### Считаем метрики

In [14]:
score = metrics.precision_score(y_test, y_pred)
false_positives, false_negatives = count_false_positives_and_negatives(y_test, y_pred)
false_positives_str = '"' + '"\n\t\t"'.join(df_test["name"][false_positives]) + '"'
false_positives_sum = sum(false_positives)

false_negatives_str = '"' + '"\n\t\t"'.join(df_test["name"][false_negatives]) + '"'
false_negatives_sum = sum(false_negatives)

print(f'Из общего числа {len(df_train_test)} строк: \n'
      f'\tЛожно-положительные: {false_positives_sum} \n\t(\n\t\t{false_positives_str}\n\t)\n'
      f'\tЛожно-отрицательные: {false_negatives_sum} \n\t(\n\t\t{false_negatives_str}\n\t)\n'
      f'\tТочность: {score}%')

Из общего числа 40000 строк: 
	Ложно-положительные: 15 
	(
		"Гриньков М. С. г. Н. Новгород"
		"Гапоненко Роза Инокентьевна"
		"Игнатова Л. В"
		"Кучуков К.Ю"
		"Цуканова Л. А"
		"Пономаренко А. В"
		"Набиев Салман Абдурахманович"
		"Глазева Н.А"
		"Бачиев М. Ю"
		"Кадеш В. Г. г. Н.Новгород"
		"Ананян С. С"
		"Садыкова Фарида Фаизовна"
		"Камаева Н. Б. г. Н. Новгород"
		"Мелькина И А"
		"Качурин Д. Н"
	)
	Ложно-отрицательные: 5 
	(
		"КПК "Общедоступный кредитъ-Мста""
		"КФХ "Мелихов и сыновья""
		"Штепо Андрей Викторович"
		"Глава Кфх Володина Валентина Михайловна"
		"ТОС Стеклозавод"
	)
	Точность: 0.9979983987189752%
